In [ ]:
import mne

files = ["/home/user/sleep-stage/rmlogic_data/data/REMLOGIC HASTA VERİLERİ/2d42885b-ba49-4c21-9e8f-4963349f252c/Traces.edf",
         "/home/user/sleep-stage/rmlogic_data/data/REMLOGIC HASTA VERİLERİ/2d42885b-ba49-4c21-9e8f-4963349f252c/Traces_annotations.edf"]

# PSG verisini ve anotasyonları oku
psg_data = mne.io.read_raw_edf(files[0], preload=True)  # PSG verisi
annotations = mne.read_annotations(files[1])  # Uyku evreleri anotasyonları

# Anotasyonları PSG verisine ekleyelim
psg_data.set_annotations(annotations)

# PSG verisinin örnekleme frekansı
sfreq = psg_data.info['sfreq']

Extracting EDF parameters from /home/user/sleep-stage/rmlogic_data/data/REMLOGIC HASTA VERİLERİ/2d42885b-ba49-4c21-9e8f-4963349f252c/Traces.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5679103  =      0.000 ... 22183.996 secs...


/tmp/ipykernel_3030637/2445920671.py:11: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  psg_data.set_annotations(annotations)


: 

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

# 30 saniyelik epoch uzunluğu
epoch_length = 30  # saniye
samples_per_epoch = int(epoch_length * sfreq)  # Her epoch'un örnek sayısı

# Sinyali 30 saniyelik epoch'lara ayıralım
X_epochs = []
n_epochs = len(annotations)  # Toplam epoch sayısı

for i in range(n_epochs):
    start_sample = i * samples_per_epoch
    end_sample = start_sample + samples_per_epoch
    
    if end_sample <= psg_data.n_times:  # Veri sınırlarını aşmamaya dikkat et
        X_epochs.append(psg_data.get_data()[:, start_sample:end_sample])

# Listeyi NumPy dizisine çevir
X_epochs = np.array(X_epochs)  # (n_epochs, n_channels, samples_per_epoch)

# Etiketleri çıkar
y = [annot['description'] for annot in annotations]


In [ ]:
def extract_features(epoch):
    """Her epoch için özellik çıkaran fonksiyon"""
    return {
        "mean": np.mean(epoch, axis=1),       # Kanal bazında ortalama
        "std": np.std(epoch, axis=1),         # Kanal bazında standart sapma
        "skewness": skew(epoch, axis=1),      # Kanal bazında çarpıklık
        "kurtosis": kurtosis(epoch, axis=1),  # Kanal bazında basıklık
        "power": np.sum(epoch**2, axis=1),    # Kanal bazında güç
    }

# Tüm epoch'lar için özellikleri çıkar
feature_list = [extract_features(epoch) for epoch in X_epochs]

# Özellikleri dataframe formatına getir
df_features = pd.DataFrame(feature_list)

# Anotasyonları dataframe'e ekle
df_features["sleep_stage"] = y


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

# Bağımsız ve bağımlı değişkenleri ayır
X = df_features.drop(columns=["sleep_stage"])
y = df_features["sleep_stage"]

# Eğitim ve test verisine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli eğitme
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# SHAP analizi
explainer = shap.Explainer(clf, X_train)
shap_values = explainer(X_test)

# SHAP grafiğini çiz
shap.summary_plot(shap_values, X_test)
